In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact
from scipy.signal import freqz
%matplotlib inline

In [5]:
audio_data = pd.read_csv("audio_features.csv")
metadata_df = pd.read_csv("metadata.csv")

In [9]:
signal = audio_data["AE_spindle_case001"] # Choose a signal to analyze

In [ ]:
w1 = 0.1*np.pi
w2 = 0.2*np.pi
delta_w = abs(w1-w2)
N_least = np.ceil(2*np.pi/delta_w)
print(N_least)

N = len(signal)
fs = 250 # Hz

filters = ({"name":"rect", "function":np.ones}, {"name":"hanning", "function":np.hanning}, {"name":"kaiser", "function":np.kaiser})
def sTFT(time_seg, filter_index):
    Nwindow         = N//time_seg
    time_freq_image = np.zeros((N//2, time_seg))
    for time_index in range(time_seg):
        signal_pad = np.zeros(N, )
        if filter_index<2:
            signal_pad[:Nwindow] = signal[time_index*Nwindow:(time_index+1)*Nwindow]*filters[filter_index]["function"](Nwindow)
        else:
            signal_pad[:Nwindow] = signal[time_index*Nwindow:(time_index+1)*Nwindow]*filters[filter_index]["function"](Nwindow, 10)
        
        signal_freq     = np.fft.fft(signal_pad)[:N//2]
        signal_freq_max = np.max(np.abs(signal_freq))
        signal_freq     = np.abs(signal_freq)/signal_freq_max
        time_freq_image[:, time_index] = 20*np.log10(signal_freq)
    
    
    w, h = plt.figaspect(0.4)
    plt.figure(figsize=(w, h))
    plt.imshow(time_freq_image[::-1, :], cmap="gray", extent=(0, N/fs*1000, 0, fs//2), aspect='auto', clim=(-60, 0))
    plt.title("time-frequency analysis, {:s} window, window size = {:d}".format(filters[filter_index]["name"], Nwindow))
    plt.xlabel("time, ms")
    plt.ylabel("frequency, Hz")
    plt.show()
    
interact(sTFT, time_seg=widgets.IntSlider(min=1, max=50,step=1
                                          ,value=1), filter_index=widgets.IntSlider(min=0, max=2,step=1,value=0));

20.0


interactive(children=(IntSlider(value=1, description='time_seg', max=50, min=1), IntSlider(value=0, descriptio…